# フラクタル

樹木の枝の付き方、雪の結晶の形、海岸線や山の稜線の形など、自然界にはその一部分を拡大すると元の形と同じようなものが現れる自己相似（フラクタル；fractal）という性質を持つものが多い<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)。今回は、いろいろなフラクタル図形を描画してみよう。

## 今回のねらい

- 写像の繰り返しによって定義される簡単な系のシミュレーションができるようになる。
- （心に余裕があれば）フラクタルアートを楽しむ。

## コッホ曲線（再帰処理）

有限の長さの線分を3等分し、分割した2点で正三角形を描いて底辺を消す。この操作を何度も繰り返してできあがるのがコッホ曲線（Koch curve）である（図1）。

![コッホ曲線](https://tueda.github.io/PS2022SS/notebooks/images/koch.png)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図1. コッホ曲線

<!-- textlint-enable -->

![線分の分割](https://tueda.github.io/PS2022SS/notebooks/images/koch_geom.png)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図2. 線分の分割

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図2のように、ある線分が点$P_1$と$P_2$によって与えられたとき、点$A$、$B$、$C$をどう求めるかは単純に平面幾何学の問題である。いま、

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length,ja-engineering-paper/use-si-units -->

$$
\vec{d} = \overrightarrow{P_1 P_2} = 
\begin{pmatrix}
x_2 - x_1 \\
y_2 - y_1
\end{pmatrix} ,
\tag{1}
$$

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

とし、$\vec{d}$を反時計回りに$90^\circ$回転させたものを$\vec{d}_\perp$としよう。$\vec{d}_\perp$は$\vec{d}$に回転行列を掛けることで得られる。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\vec{d}_\perp = 
\begin{pmatrix}
\displaystyle \cos\left(\frac{\pi}{2}\right) &
\displaystyle -\sin\left(\frac{\pi}{2}\right) \\
\displaystyle \sin\left(\frac{\pi}{2}\right) &
\displaystyle \phantom{-} \cos\left(\frac{\pi}{2}\right)
\end{pmatrix}
\vec{d} =
\begin{pmatrix}
y_1 - y_2 \\
x_2 - x_1
\end{pmatrix} .
\tag{2}
$$

<!-- textlint-enable -->

すると、図2から明らかなように、以下のように点$A$、$B$、$C$の位置を求めることができる。

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{align}
A &= P_1 + \frac{1}{3} \vec{d} ,
\tag{3} \\
B &= P_1 + \frac{1}{2} \vec{d} + \frac{\sqrt{3}}{6} \vec{d}_\perp ,
\tag{4} \\
C &= P_1 + \frac{2}{3} \vec{d} .
\tag{5}
\end{align}
$$

<!-- textlint-enable -->

いま描きたいコッホ曲線は階層的な構造を持っているので、関数の再帰呼び出しと相性がよい。線分を分割する関数を作り、その関数からそれ自身を再帰的に呼び出すことによって、線分の分割を繰り返せばよい<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3)。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from math import sqrt

max_level = 6  # 再帰の深さ

# 折れ線グラフを描くための座標を保存するリスト（グローバル変数として使う）
xpoints = []
ypoints = []


def koch(x1, y1, x2, y2, level):  # (x1,y1)から(x2,y2)の間のコッホ曲線
    if level == 0:
        # levelが0であれば点を追加する
        if len(xpoints) == 0:
            # リストが空のとき（つまり最初のみ）
            xpoints.append(x1)
            ypoints.append(y1)
        xpoints.append(x2)
        ypoints.append(y2)
    else:
        # levelが0より大きければ、点A、B、Cの位置を求め、
        # 4つの線分に分割する(level - 1を渡して再帰)
        dx = x2 - x1
        dy = y2 - y1

        xa = x1 + 1 / 3 * dx
        ya = y1 + 1 / 3 * dy

        xb = x1 + 1 / 2 * dx - sqrt(3) / 6 * dy
        yb = y1 + 1 / 2 * dy + sqrt(3) / 6 * dx

        xc = x1 + 2 / 3 * dx
        yc = y1 + 2 / 3 * dy

        koch(x1, y1, xa, ya, level - 1)
        koch(xa, ya, xb, yb, level - 1)
        koch(xb, yb, xc, yc, level - 1)
        koch(xc, yc, x2, y2, level - 1)


# 実際にkoch関数を呼ぶ
koch(0, 0, 1, 0, max_level)

# 折れ線グラフを描く
fig, ax = plt.subplots()
ax.plot(xpoints, ypoints)
ax.set_aspect("equal")
plt.show()

## 練習問題

<!-- textlint-disable ja-technical-writing/sentence-length,ja-technical-writing/max-comma -->

(1) 上のプログラムでは$(0, 0) \to (1, 0)$の間にコッホ曲線を描いた。これに加えて、$(1, 0) \to (1 / 2, -\sqrt{3}/2)$、$(1 / 2, -\sqrt{3}/2) \to (0, 0)$の間の2つのコッホ曲線を追加してみよう（つまり、`koch`関数を2回追加で呼ぶ）。できた図形はコッホ雪片（Koch snowflake）と呼ばれる<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4)。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

(2) コッホ曲線は、その一部を拡大しても同じ形が現れるという、フラクタル構造を持っている。これを確認してみよう。[Axesオブジェクト](https://matplotlib.org/stable/api/axes_api.html#the-axes-class)の[set_xlimメソッド](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_xlim.html)や[set_ylimメソッド](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_ylim.html)を使って、横軸と縦軸の範囲を指定してやればよい（たとえば、$0.10 \le x \le 0.23$と$0.05 \le y \le 0.10$）。

<!-- textlint-enable -->

## コッホ曲線（カオスゲーム）

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

今度はコッホ曲線を違った方法で描いてみよう。次のような2つの2次元写像を繰り返すことを考える：

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} x_n + \frac{\sqrt{3}}{6} y_n , \\
y_{n+1} = \displaystyle \frac{\sqrt{3}}{6} x_n - \frac{1}{2} y_n ,
\end{cases}
\tag{6} \\
\text{②} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} x_n - \frac{\sqrt{3}}{6} y_n + \frac{1}{2} , \\
y_{n+1} = \displaystyle - \frac{\sqrt{3}}{6} x_n - \frac{1}{2} y_n + \frac{\sqrt{3}}{6} .
\end{cases}
\tag{7}
\end{align}
$$

<!-- textlint-enable -->

ただし、各ステップで写像①と②をそれぞれ確率$1/2$で、どちらか1つを選んで適用するものとする。このようにしてできる点の集合を**散布図**によって描画する。初期位置は$(x_0,y_0) = (0, 0)$としてみよう<a name="cite_ref-5"></a>[<sup>[5]</sup>](#cite_note-5)。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
from math import sqrt

n = 100000  # ステップ数

xpoints = []  # 各ステップでの位置を保存するリスト
ypoints = []

# 初期位置
x = 0
y = 0
xpoints.append(x)
ypoints.append(y)

for _ in range(n):
    # 2つの写像のどちらかをランダムに選んで適用する
    r = random.random()
    if r < 0.5:
        x_new = 1 / 2 * x + sqrt(3) / 6 * y
        y_new = sqrt(3) / 6 * x - 1 / 2 * y
    else:
        x_new = 1 / 2 * x - sqrt(3) / 6 * y + 1 / 2
        y_new = -sqrt(3) / 6 * x - 1 / 2 * y + sqrt(3) / 6

    x = x_new
    y = y_new

    # 各ステップでの位置をリストに追加
    xpoints.append(x)
    ypoints.append(y)

# 散布図を描く
fig, ax = plt.subplots()
ax.scatter(xpoints, ypoints, s=0.01)
ax.set_aspect("equal")
plt.show()

このようにしてフラクタル図形を描く方法はカオスゲーム（chaos game）と呼ばれる<a name="cite_ref-6"></a>[<sup>[6]</sup>](#cite_note-6)。

## 練習問題

<!-- textlint-disable jtf-style/4.3.1.丸かっこ（） -->

(3) 式<!-- eqref -->(6)と<!-- eqref -->(7)の係数を1つでも変更すると違う図形となることを確認してみよう（演習課題(2)参照）。

<!-- textlint-enable -->

## シェルピンスキーのギャスケット

次の3つの写像を使ってカオスゲームを行う。

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} x_n  , \\
y_{n+1} = \displaystyle \frac{1}{2} y_n ,
\end{cases}
\tag{8} \\
\text{②} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} (x_n + 2) , \\
y_{n+1} = \displaystyle \frac{1}{2} y_n ,
\end{cases}
\tag{9} \\
\text{③} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} (x_n + 1) , \\
y_{n+1} = \displaystyle \frac{1}{2} (y_n + 1) .
\end{cases}
\tag{10}
\end{align}
$$

<!-- textlint-enable -->

ただし、①から③の写像をそれぞれ確率$1/3$で選ぶ。初期位置は$(x_0,y_0) = (1/2, 1/2)$としてみよう。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

n = 100000  # ステップ数

xpoints = []  # 各ステップでの位置を保存するリスト
ypoints = []

# 初期位置
x = 1 / 2
y = 1 / 2
xpoints.append(x)
ypoints.append(y)

for _ in range(n):
    # 3つの写像のどれかを適用する
    r = random.random()
    if r < 1 / 3:
        x_new = 1 / 2 * x
        y_new = 1 / 2 * y
    elif r < 2 / 3:
        x_new = 1 / 2 * (x + 2)
        y_new = 1 / 2 * y
    else:
        x_new = 1 / 2 * (x + 1)
        y_new = 1 / 2 * (y + 1)

    x = x_new
    y = y_new

    # 各ステップでの位置をリストに追加
    xpoints.append(x)
    ypoints.append(y)

# 散布図を描く
fig, ax = plt.subplots()
ax.scatter(xpoints, ypoints, s=0.01)
ax.set_aspect("equal")
plt.show()

この図形はシェルピンスキーのギャスケット（Sierpiński gasket）と呼ばれる<a name="cite_ref-7"></a>[<sup>[7]</sup>](#cite_note-7)<a name="cite_ref-8"></a>[<sup>[8]</sup>](#cite_note-8)。

## バーンズリーのシダ

次の4つの写像を使ってカオスゲームを行う。

<!-- textlint-disable ja-technical-writing/sentence-length,ja-engineering-paper/use-si-units -->

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.85 x_n + 0.04 y_n , \\
y_{n+1} = - 0.04 x_n +0.85 y_n + 1.60,
\end{cases}
\tag{11} \\
\text{②} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.20 x_n - 0.26 y_n , \\
y_{n+1} = \phantom{-} 0.23 x_n + 0.22 y_n + 1.60,
\end{cases}
\tag{12} \\
\text{③} \,
&\begin{cases}
x_{n+1} = - 0.15 x_n + 0.28 y_n , \\
y_{n+1} = \phantom{-} 0.26 x_n + 0.24 y_n + 0.44,
\end{cases}
\tag{13} \\
\text{④} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.00 , \\
y_{n+1} = \phantom{-} 0.16 y_n .
\end{cases}
\tag{14}
\end{align}
$$

<!-- textlint-enable -->

ただし、①から④の写像を選ぶ確率はそれぞれ① 0.85、② 0.07、③ 0.07、④ 0.01とし、初期位置は $(x_0,y_0) = (0, 0)$ とする。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

n = 100000  # ステップ数

xpoints = []  # 各ステップでの位置を保存するリスト
ypoints = []

# 初期位置
x = 0
y = 0
xpoints.append(x)
ypoints.append(y)

for _ in range(n):
    # 4つの写像のどれかを適用する
    r = random.random()
    if r < 0.85:
        x_new = +0.85 * x + 0.04 * y
        y_new = -0.04 * x + 0.85 * y + 1.60
    elif r < 0.92:
        x_new = +0.20 * x - 0.26 * y
        y_new = +0.23 * x + 0.22 * y + 1.60
    elif r < 0.99:
        x_new = -0.15 * x + 0.28 * y
        y_new = +0.26 * x + 0.24 * y + 0.44
    else:
        x_new = 0
        y_new = 0.16 * y

    x = x_new
    y = y_new

    # 各ステップでの位置をリストに追加
    xpoints.append(x)
    ypoints.append(y)

# 散布図を描く
fig, ax = plt.subplots()
ax.scatter(xpoints, ypoints, s=0.01, c="green")
ax.set_aspect(0.5)
plt.show()

この図形はバーンズリーのシダ（Barnsley fern）と呼ばれる。

## アポロニウスの窓

もう少し複雑な写像（非線形写像）を用いることもできる<a name="cite_ref-9"></a>[<sup>[9]</sup>](#cite_note-9)。今度は次の4つの写像を使ってカオスゲームを行う<a name="cite_ref-10"></a>[<sup>[10]</sup>](#cite_note-10)。

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = \displaystyle x_n  , \\
y_{n+1} = \displaystyle - y_n ,
\end{cases}
\tag{15} \\
\text{②} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{x_n}{16 x_n^2 + (4 y_n - 1)^2} , \\
y_{n+1} = \displaystyle \frac{4 x_n^2 + y_n (4 y_n - 1)}{16 x^2 + (4 y - 1)^2} ,
\end{cases}
\tag{16} \\
\text{③} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{x (x - 1) + (y - 1)^2}{(x - 1)^2 + (y - 1)^2} , \\
y_{n+1} = \displaystyle \frac{(x - 1)^2 + y (y - 1)}{(x - 1)^2 + (y - 1)^2} ,
\end{cases}
\tag{17} \\
\text{④} \,
&\begin{cases}
x_{n+1} = \displaystyle - \frac{x (x + 1) + (y - 1)^2}{(x + 1)^2 + (y - 1)^2} , \\
y_{n+1} = \displaystyle \frac{(x + 1)^2 + y (y - 1)}{(x + 1)^2 + (y - 1)^2} .
\end{cases}
\tag{18}
\end{align}
$$

<!-- textlint-enable -->

ただし、①から④の写像をそれぞれ確率$1/4$で選ぶ。初期位置は$(x_0,y_0) = (1/2, 1/2)$とする。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

n = 100000  # ステップ数

xpoints = []  # 各ステップでの位置を保存するリスト
ypoints = []

# 初期位置
x = 1 / 2
y = 1 / 2
xpoints.append(x)
ypoints.append(y)

for _ in range(n):
    # 4つの写像のどれかを適用する
    r = random.random()
    if r < 1 / 4:
        x_new = x
        y_new = -y
    elif r < 1 / 2:
        x_new = x / (16 * x**2 + (4 * y - 1) ** 2)
        y_new = (4 * x**2 + y * (4 * y - 1)) / (16 * x**2 + (4 * y - 1) ** 2)
    elif r < 3 / 4:
        x_new = (x * (x - 1) + (y - 1) ** 2) / ((x - 1) ** 2 + (y - 1) ** 2)
        y_new = ((x - 1) ** 2 + y * (y - 1)) / ((x - 1) ** 2 + (y - 1) ** 2)
    else:
        x_new = -(x * (x + 1) + (y - 1) ** 2) / ((x + 1) ** 2 + (y - 1) ** 2)
        y_new = ((x + 1) ** 2 + y * (y - 1)) / ((x + 1) ** 2 + (y - 1) ** 2)

    x = x_new
    y = y_new

    # 各ステップでの位置をリストに追加
    xpoints.append(x)
    ypoints.append(y)

# 散布図を描く
fig, ax = plt.subplots()
ax.scatter(xpoints, ypoints, s=0.01)
ax.set_aspect("equal")
plt.show()

この図形はアポロニウスの窓（Apollonian window）と呼ばれる。

## 演習課題

(1) 次の4つの写像を使ったカオスゲームを考える。

<!-- textlint-disable ja-technical-writing/sentence-length,ja-engineering-paper/use-si-units -->

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = 0.8 x_n , \\
y_{n+1} = 0.8 y_n ,
\end{cases}
\tag{19} \\
\text{②} \,
&\begin{cases}
x_{n+1} = 0.5 x_n + 0.000 , \\
y_{n+1} = 0.5 y_n + 0.315 ,
\end{cases}
\tag{20} \\
\text{③} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.355 x_n - 0.355 y_n - 0.163 , \\
y_{n+1} = \phantom{-} 0.355 x_n + 0.355 y_n + 0.062 ,
\end{cases}
\tag{21} \\
\text{④} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.355 x_n + 0.355 y_n + 0.163 , \\
y_{n+1} = - 0.355 x_n + 0.355 y_n + 0.062 .
\end{cases}
\tag{22}
\end{align}
$$

<!-- textlint-enable -->

ただし、①から④の写像を選ぶ確率はそれぞれ① 0.5、② 0.168、③ 0.166、④ 0.166、初期位置は $(x_0,y_0) = (0, 0)$ とする。これらの写像を繰り返して得られる点の集合よりフラクタル図形を描画せよ。点の色は（そのフラクタル図形から想像される色などに）適当に変えてもよい。

In [ ]:
# 課題解答10.1  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import random

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

(2) アポロニウスの窓の場合を除くと、今回用いた写像の一般形は、線形写像に平行移動を加えた

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{cases}
x_{n+1} = a_{11} x_n + a_{12} y_n + b_1 , \\
y_{n+1} = a_{21} x_n + a_{22} y_n + b_2 ,
\end{cases}
\tag{23}
$$

<!-- textlint-enable -->

であり、アフィン写像（affine map）と呼ばれる。コッホ曲線を描画した例題プログラムのように、いま**2種類**のアフィン写像を用い、その係数を適宜修正してオリジナルの**フラクタル図形**を描画してみよう。ただし、2つの写像を選ぶ確率はそれぞれ**1/2ずつに固定**することにする。いくつか試して、気に入ったデザインのもののを提出せよ。なお、[scatterメソッド](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.scatter.html)の`c`オプションや`cmap`オプションをうまく指定して、点の色を変えても面白いだろう<a name="cite_ref-11"></a>[<sup>[11]</sup>](#cite_note-11)。

In [ ]:
# 課題解答10.2  <-- 提出する際に、この行を必ず含めること。


%matplotlib inline
import matplotlib.pyplot as plt
import random

# 使いたければmathモジュールやnumpyモジュールの関数を適宜インポートせよ。

## 発展課題

<!-- textlint-disable jtf-style/1.1.1.本文 -->

余裕があればやってみてください。

<!-- textlint-enable -->

(3) 今回カオスゲームによって描いたシェルピンスキーのギャスケットを、ほかの方法で描いてみよう。

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

いま、$n \times n = n^2$個の要素を持った2次元リスト`a`を、

<!-- textlint-enable -->

In [ ]:
n = 500

a = []
for _ in range(n):
    a.append([0] * n)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

あるいは、

<!-- textlint-enable -->

In [ ]:
n = 500

a = [[0] * n for _ in range(n)]

のようにして作る（どちらでもよい）<a name="cite_ref-12"></a>[<sup>[12]</sup>](#cite_note-12)。すると、`a`の要素へのアクセスは`a[i][j]`のように書ける。ただし、$i$と$j$の範囲は$0 \le i \le n - 1$、$0 \le j \le n - 1$である。`a`の要素はすべて`0`で初期化されている。

まず、`a`の各要素を次式に従って設定せよ<a name="cite_ref-13"></a>[<sup>[13]</sup>](#cite_note-13)。

$$
a_{i,j} =
\begin{cases}
1, & \text{for } 0 \le i \le n - 1 \text{ and } j = 0, \\
1, & \text{for } i = 0 \text{ and } 0 \le j \le n - 1, \\
a_{i-1,j} + a_{i,j-1}, & \text{otherwise}. 
\end{cases}
\tag{24}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

それができたら、$0 \le i + j \le n - 1$を満たす`a`のすべての要素`a[i][j]`に対し、もし**その値が奇数なら**ば、

<!-- textlint-enable -->

$$
x_{i,j} = i - j, \qquad
y_{i,j} = - i - j, \qquad
\tag{25}
$$

で表される座標$(x_{i,j},y_{i,j})$に点を打つことで散布図を作れ。$n$の値は$500$程度としてみよう。

In [ ]:
# 課題解答10.3  <-- 提出する際に、この行を必ず含めること。


%matplotlib inline
import matplotlib.pyplot as plt

# 2次元リストを作成、各要素を設定し、それに基づいて散布図を描いてください。

## 参考：アフィン変換

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

式<!-- eqref -->(23)は、拡大係数行列（augmented matrix）を用いて表した

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{pmatrix}
x_{n+1} \\
y_{n+1} \\
1
\end{pmatrix}
=
\begin{pmatrix}
a_{11} & a_{12} & b_1 \\
a_{21} & a_{22} & b_2 \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
x_n \\
y_n \\
1
\end{pmatrix} ,
\tag{26}
$$

<!-- textlint-enable -->

と等価である。このような行列は（正則であれば）群を成し、座標変換を表すのに都合がよい。任意の2次元アフィン変換は回転、拡大縮小、平行移動を表す行列の積として表される<a name="cite_ref-14"></a>[<sup>[14]</sup>](#cite_note-14)<a name="cite_ref-15"></a>[<sup>[15]</sup>](#cite_note-15)。

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{align}
\text{Rotation}(\theta) &=
\begin{pmatrix}
\cos(\theta) & - \sin(\theta) & 0 \\
\sin(\theta) & \phantom{-} \cos(\theta) & 0 \\
0 & 0 & 1
\end{pmatrix} ,
\tag{27} \\
\text{Scaling}(s_x, s_y) &=
\begin{pmatrix}
s_x & 0 & 0 \\
0 & s_y & 0 \\
0 & 0 & 1
\end{pmatrix} ,
\tag{28} \\
\text{Translation}(t_x, t_y) &=
\begin{pmatrix}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1
\end{pmatrix} .
\tag{29}
\end{align}
$$

<!-- textlint-enable -->

3次元空間への拡張も容易である。複数のアフィン変換の合成も行列の積によって1つにまとめることができるなど便利であり、コンピューターグラフィックスの分野などで多用される。

<!-- textlint-disable jtf-style/4.3.1.丸かっこ（） -->

行列の積によるアフィン変換の合成の例として、式<!-- eqref -->(6)と<!-- eqref -->(7)の写像はそれぞれ、次のような行列の積で表される。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{pmatrix}
\displaystyle \cos\left(\frac{\pi}{6}\right) \vphantom{\frac{1}{\sqrt{3}}} &
\displaystyle -\sin\left(\frac{\pi}{6}\right) &
0 \\
\displaystyle \sin\left(\frac{\pi}{6}\right) \vphantom{\frac{1}{\sqrt{3}}} &
\displaystyle \phantom{-} \cos\left(\frac{\pi}{6}\right) & 
0 \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
\displaystyle \frac{1}{\sqrt{3}} & 0 & 0 \\
0 & \displaystyle - \frac{1}{\sqrt{3}} & 0 \\
0 & 0 & 1
\end{pmatrix}
=
\begin{pmatrix}
\displaystyle \frac{1}{2} &
\displaystyle \frac{\sqrt{3}}{6} & 0 \\
\displaystyle \frac{\sqrt{3}}{6} &
\displaystyle - \frac{1}{2} & 0 \\
0 & 0 & 1
\end{pmatrix} ,
\tag{30}
$$

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

$$
\begin{pmatrix}
1 &
0 &
\displaystyle \frac{1}{2} \vphantom{\frac{1}{\sqrt{3}}} \\
0 &
1 & 
\displaystyle \frac{\sqrt{3}}{6} \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
\displaystyle \cos\left(-\frac{\pi}{6}\right) \vphantom{\frac{1}{\sqrt{3}}} &
\displaystyle -\sin\left(-\frac{\pi}{6}\right) &
0 \\
\displaystyle \sin\left(-\frac{\pi}{6}\right) \vphantom{\frac{1}{\sqrt{3}}} &
\displaystyle \phantom{-} \cos\left(-\frac{\pi}{6}\right) & 
0 \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
\displaystyle \frac{1}{\sqrt{3}} & 0 & 0 \\
0 & \displaystyle - \frac{1}{\sqrt{3}} & 0 \\
0 & 0 & 1
\end{pmatrix}
=
\begin{pmatrix}
\displaystyle \frac{1}{2} &
\displaystyle - \frac{\sqrt{3}}{6} &
\displaystyle \frac{1}{2} \\
\displaystyle - \frac{\sqrt{3}}{6} &
\displaystyle - \frac{1}{2} &
\displaystyle \frac{\sqrt{3}}{6} \\
0 & 0 & 1
\end{pmatrix} .
\tag{31}
$$

<!-- textlint-enable -->

<!-- textlint-disable ja-engineering-paper/use-si-units -->

これらは、図2の$\triangle P_1 P_2 B$を、それぞれ$\triangle P_1 B A$と$\triangle B P_2 C$に移す写像となっている<a name="cite_ref-16"></a>[<sup>[16]</sup>](#cite_note-16)。

<!-- textlint-enable -->

## 脚注

<a name="cite_note-1"></a>1.&nbsp;[^](#cite_ref-1)
自己相似性を表すものとしてフラクタルという概念を導入し数学的に厳密な定義を与えたのはブヌワ・マンデルブロットである。

<a name="cite_note-2"></a>2.&nbsp;[^](#cite_ref-2)
分割操作を無限に繰り返して得られるのがコッホ曲線である。しかし、計算機上では（無限回繰り返すと無限の時間がかかってしまうので）有限回の繰り返しで止めて、近似的な曲線を求めることになる。

なお、（無限に分割操作を繰り返して得られる）コッホ曲線は、いたるところで微分不可能である（接線が引けない）。

<a name="cite_note-3"></a>3.&nbsp;[^](#cite_ref-3)
このプログラムでは、グローバル変数として`xpoints`と`ypoints`というリストを用意し、関数`koch`の中でこれらの`append`メソッドを呼ぶことで点を追加している。代わりに、`xpoints`と`ypoints`を関数`koch`の引数としておいてもよい（関数の引数は参照の値渡しで渡されるということさえ理解していれば）。これくらい短いプログラムであれば問題は起こりづらいが、基本的に**グローバル変数は悪**（[global variables are evil](https://www.google.com/search?q=global+variables+are+evil)）である。

<!-- textlint-disable ja-technical-writing/sentence-length,ja-technical-writing/max-ten -->

<a name="cite_note-4"></a>4.&nbsp;[^](#cite_ref-4)
コッホ曲線の分割操作を1回行うと、線分の全長は$4/3$倍になる。これを無限回繰り返すことになるので、コッホ曲線の長さは無限大である。よって、コッホ曲線を3つつないだ図形であるコッホ雪片の周の長さも無限大となる。一方、面積については、$n$回目の操作によって面積が$(1/3)^{2n} \sqrt{3} \, / 4$ である$3 \times 4^{n-1}$個の正三角形の分だけ増えるから、もとの三角形の面積$\sqrt{3}/4$とこれらを（無限個）すべて足し合わせて、$2\sqrt{3} \, / 5$と求まる。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/no-doubled-conjunction -->

このような、2次元において有限の図形中に定義された無限の長さを持つ曲線の中でも、極端な例として[ペアノ曲線](https://ja.wikipedia.org/wiki/%E3%83%9A%E3%82%A2%E3%83%8E%E6%9B%B2%E7%B7%9A)が知られている。ペアノ曲線は正方形を埋め尽くす曲線である。通常、始点と終点を持った曲線上の位置は、1つの自由度によって指定できる。正方形の中の位置は2つの自由度（たとえば$x$と$y$）によって指定できる。このような自由度のことをナイーブには「次元」と呼ぶが、ペアノ曲線の発見により平面と曲線の自由度は同じだと言えてしまい、さらに正確な「次元」の定義が必要となった。たとえばペアノ曲線のハウスドルフ次元は$2$、コッホ曲線のハウスドルフ次元は$\log_3(4) \approx 1.26186$となる。もっと知りたければ、たとえば次を参照せよ。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/max-comma,ja-engineering-paper/unify-kuten-and-touten,ja-spacing/ja-space-between-half-and-full-width,jtf-style/1.2.1.句点(。)と読点(、),jtf-style/3.1.1.全角文字と半角文字の間 -->

- 高安秀樹, フラクタルの物理, 物性研究 44(6), 885-981, 1985, http://hdl.handle.net/2433/91795.

<!-- textlint-enable -->

<a name="cite_note-5"></a>5.&nbsp;[^](#cite_ref-5)
十分にたくさんの点を打つと、描かれる図は（始めの数個を除いては）初期位置に依存しない。しかし筆者は描かれる図をすでに知っているので、ここではその図の主要な部分の1点からカオスゲームを始めている。

<!-- textlint-disable jtf-style/4.3.1.丸かっこ（） -->

<a name="cite_note-6"></a>6.&nbsp;[^](#cite_ref-6)
式<!-- eqref -->(6)、<!-- eqref -->(7)のような複数の写像（通常は縮小写像）の和集合によって定義される系を反復関数系（iterated function system）と呼ぶ。カオスゲームでは、各ステップにおいて、反復関数系に含まれる写像の中から確率的に1つを選択することにより系を発展させている。

<!-- textlint-enable -->

<!-- textlint-disable ja-engineering-paper/prh,ja-technical-writing/ja-no-weak-phrase -->

<a name="cite_note-7"></a>7.&nbsp;[^](#cite_ref-7)
次回もまたこの図形にお目にかかるかもしれない。

<!-- textlint-enable -->

<a name="cite_note-8"></a>8.&nbsp;[^](#cite_ref-8)
出来上がった図形を見ても想像がつくとおり、シェルピンスキーのギャスケットも再帰的な手続きによって定義でき、無限回の分割操作によって得られる面積の極限は$0$となる。

<a name="cite_note-9"></a>9.&nbsp;[^](#cite_ref-9)
カオスゲーム（および反復関数系）に対して、ほかの非線形写像を用いる例は、たとえば[これ](https://en.wikipedia.org/wiki/Fractal_flame)。

<a name="cite_note-10"></a>10.&nbsp;[^](#cite_ref-10)
Jerzy Kocik, *Apollonian Window*, http://lagrange.math.siu.edu/Kocik/apollo/achaos/chaos-explain.htm.

<a name="cite_note-11"></a>11.&nbsp;[^](#cite_ref-11)
たとえば、各点の$x$座標と$y$座標から適当な実数値を生成する関数を定義しておく。
```python
def make_color(x, y):
    return x ** 2 + y ** 2
```
これにより、各点の色に対応する色を決定する。
```python
    xpoints.append(x)
    ypoints.append(y)
    colors.append(make_color(x, y))
```
あとは、[`scatter`メソッド](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.scatter.html)を呼び出す際に、`c`とともに`cmap`を指定すればよい。
```python
ax.scatter(xpoints, ypoints, s=0.01, c=colors, cmap="viridis")
```
あらかじめ用意されているカラーマップ（`cmap`）の種類については[こちら](https://matplotlib.org/stable/tutorials/colors/colormaps.html)を参照せよ。

<a name="cite_note-12"></a>12.&nbsp;[^](#cite_ref-12)
次のように`a`を作ると、ここで意図しているものにはならないので注意。
```python
a = [[0] * n] * n  # 内側にある、同一リストへの参照をn個持ったリスト
```

<a name="cite_note-13"></a>13.&nbsp;[^](#cite_ref-13)
この$a_{i,j}$の定義は[パスカルの三角形](https://ja.wikipedia.org/wiki/%E3%83%91%E3%82%B9%E3%82%AB%E3%83%AB%E3%81%AE%E4%B8%89%E8%A7%92%E5%BD%A2)と同等のものである。

<!-- textlint-disable ja-technical-writing/sentence-length -->

<a name="cite_note-14"></a>14.&nbsp;[^](#cite_ref-14)
剪断（せんだん）と呼ばれる写像もあるが、これは特異値分解（singular value decomposition；SVD）という行列分解によって、いくつかの拡大縮小と回転の積へと分解できる。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length,ja-technical-writing/max-ten,jtf-style/4.3.1.丸かっこ（） -->

<a name="cite_note-15"></a>15.&nbsp;[^](#cite_ref-15)
これを見ていると、行列式$\displaystyle \begin{vmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{vmatrix}$の絶対値が$1$より大きいような写像を演習課題(2)に使うとそれは拡大変換を含んでいるので、原点からの距離がどんどん大きくなってしまってまずそうだ（固定点がなく収束しない）、ということが分かるだろう。

<!-- textlint-enable -->

<!-- textlint-disable
ja-technical-writing/ja-no-mixed-period
-->

<a name="cite_note-16"></a>16.&nbsp;[^](#cite_ref-16)
すぐわかるように、シェルピンスキーのギャスケットの式<!-- eqref -->(8)、<!-- eqref -->(9)、<!-- eqref -->(10)も、それぞれ

<!-- textlint-enable -->

$$
\begin{pmatrix}
\displaystyle \frac{1}{2} & 0 & 0 \\
0 & \displaystyle \frac{1}{2} & 0 \\
0 & 0 & 1
\end{pmatrix},
\qquad
\begin{pmatrix}
1 & 0 & 1 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
\displaystyle \frac{1}{2} & 0 & 0 \\
0 & \displaystyle \frac{1}{2} & 0 \\
0 & 0 & 1
\end{pmatrix},
\qquad
\begin{pmatrix}
1 & 0 & \displaystyle \frac{1}{2} \\
0 & 1 & \displaystyle \frac{1}{2} \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
\displaystyle \frac{1}{2} & 0 & 0 \\
0 & \displaystyle \frac{1}{2} & 0 \\
0 & 0 & 1
\end{pmatrix},
$$

<!-- textlint-disable
ja-technical-writing/sentence-length
-->

という行列で表現できる。これらは、座標$(0,0)-(2,0)-(1,1)$からなる三角形を、それぞれ$(0,0)-(1,0)-(1/2,1/2)$、$(1,0)-(2,0)-(3/2,1/2)$、$(1/2,1/2)-(3/2,1/2)-(1,1)$の、相似ではあるが縮小された三角形へと移す写像となっている。このような複数の縮小写像を使ってカオスゲームを行うと、どのように初期値を選んでも、フラクタル構造を持ったアトラクターへと収束する。

<!-- textlint-enable -->